# Import modules and load data

In [1]:
#autoload modules in notebook
%load_ext autoreload
%autoreload 2  

In [2]:
def test():
    return 1

In [3]:
type(test)

function

In [4]:
from itertools import combinations
import json
import pandas as pd
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import wandb
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import CLIP_model
import CLIP_data_load

2023-08-21 20:28:44.707917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 20:28:45.248005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

In [6]:
# Hyperparameters
train_percentage = 0.8
test_percentage = 0.2
validation_percentage = 0.15
latent_dim_imgs = 1024
latent_dim_text = 768
latent_dim_common = 512
batch_size = 8
SEED = 116
img_shape = (128,128,3)
text_input_shape = (200)
bert_model_name = 'distilbert-base-uncased' #bert-base-multilingual-uncased
max_len = 200
model_name = 'test_regularization'

# Paths
all_images_path = 'dataset/resized_train/'
project_location = '.'
all_captions_path = f'{project_location}/dataset/caption_prediction_train.csv'  
all_concept_ids_path = f'{project_location}/dataset/concept_detection_train.csv'
all_concepts_path = f'{project_location}/dataset/concepts.csv'

zip_dataset_location = f'{project_location}/dataset/resized_train.zip'
model_loc = f'{project_location}/weights/{model_name}.h5'

In [7]:
run = wandb.init(project="clip",
           entity='calonca',
           name=model_name,
           config = {
              'batch_size':batch_size,
              'latent_dim_common':latent_dim_common,
              'latent_dim_text':latent_dim_text,
              'latent_dim_imgs':latent_dim_imgs,
              'img_shape':img_shape,
              'text_input_shape':text_input_shape,
              'model_name': model_name,
           })

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alexlaconca (calonca). Use `wandb login --relogin` to force relogin


In [8]:
#Merging all dataframes toghether into ID, caption, cuis, concepts
captionsDF = pd.read_csv(all_captions_path, sep='\t')
concept_id_df = pd.read_csv(all_concept_ids_path, sep='\t')
concept_df = pd.read_csv(all_concepts_path, sep=',')

concepts = concept_df.set_index('concept').T.to_dict('list')
concept_id_df.cuis = concept_id_df.cuis.apply(lambda x: x.split(';'))
concept_id_df['concepts'] = concept_id_df.cuis.apply(lambda cuis: {concepts[cui][0] for cui in cuis})

captionsDF = pd.merge(captionsDF,concept_id_df, on="ID")
captionsDF.ID = captionsDF.ID.apply(lambda x: x+'.jpg')

In [9]:
train_val, test = train_test_split(captionsDF, test_size = test_percentage, shuffle = False, random_state = SEED)
train, val = train_test_split(train_val, test_size = validation_percentage, shuffle = False, random_state = SEED)

In [10]:
#use only a subset of the dataset, one batch
train = train[:5*batch_size]
# val = val[:batch_size]
# test = test[:batch_size]
val = train.copy()

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained(bert_model_name)

trainList = CLIP_data_load.paths_captions_emb_list(train,  all_images_path,tokenizer=tokenizer,max_len=max_len)
valList = CLIP_data_load.paths_captions_emb_list(val, all_images_path,tokenizer=tokenizer,max_len=max_len)

100%|██████████| 40/40 [00:00<00:00, 3097.20it/s]


In [12]:
train_gen = CLIP_data_load.ClipBaseGenerator(data = trainList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)
val_gen = CLIP_data_load.ClipBaseGenerator(data = valList, batch_size = batch_size, out_shape = img_shape[0:2], shuffle = True,seed=SEED)

## **Train CLIP**

In [13]:
from wandb.keras import WandbCallback
wandb_callback = WandbCallback(monitor='loss',
                               log_batch_frequency=10,
                               save_model = False,
                               )
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500,restore_best_weights=True)

In [14]:
fen_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    pooling='avg',
    input_shape = img_shape
)

bert_model = TFDistilBertModel.from_pretrained(bert_model_name)

model: CLIP_model.CLIP_base = CLIP_model.get_clip_model(
    image_input_shape = img_shape,
    text_input_shape = text_input_shape,
    text_encoder = bert_model,
    image_encoder = fen_model,
    latent_dim_imgs = latent_dim_imgs,
    latent_dim_text = latent_dim_text,
    latent_dim_common = latent_dim_common,
    train_bert = True,
    # use our loss with temperature 0.5
    loss = CLIP_model.clip_loss,
    custom_metric = CLIP_model.custom_loss_with_temp(temperature=1),
    learning_rate=1e-5,
) # type: ignore # 

2023-08-21 20:28:54.284255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 20:28:54.301982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 20:28:54.302180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
model.losses

[]

In [16]:
[m.name for m in model.metrics]

['loss', 'custom_loss']

In [17]:
#new loss temp 1 test val diff 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 1000, 
                    callbacks = [wandb_callback],
                   workers=4)

Epoch 1/1000


2023-08-21 20:29:02.348330: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-21 20:29:04.875652: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xd1a0d8b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-21 20:29:04.875679: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2023-08-21 20:29:04.879363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-21 20:29:05.001169: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 50s 1s/step - loss: 2.0850 - custom_loss: 2.3446 - val_loss: 2.0867 - val_custom_loss: 2.2644
Epoch 2/1000
3/5 [=================>............] - ETA: 1s - loss: 2.0788 - custom_loss: 2.2888

KeyboardInterrupt: 

In [ ]:
print(f"model metrics are ", model.metrics_names)


model metrics are  ['loss', 'dense_1_loss', 'dense_1_our_loss']


In [ ]:
model.metrics

In [ ]:
#new loss temp 1 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 1000, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/1000


2023-08-18 20:29:07.682272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-18 20:29:12.049434: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xd616f3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-18 20:29:12.049470: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2023-08-18 20:29:12.104093: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-18 20:29:12.401295: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 55s 1s/step - loss: 2.4634 - val_loss: 2.4805
Epoch 2/1000
5/5 [==============================] - 6s 1s/step - loss: 2.1281 - val_loss: 2.4107
Epoch 3/1000
5/5 [==============================] - 7s 1s/step - loss: 2.1306 - val_loss: 2.1877
Epoch 4/1000
5/5 [==============================] - 7s 1s/step - loss: 1.9255 - val_loss: 1.9480
Epoch 5/1000
5/5 [==============================] - 6s 1s/step - loss: 2.0381 - val_loss: 1.8749
Epoch 6/1000
5/5 [==============================] - 7s 1s/step - loss: 1.6972 - val_loss: 1.8505
Epoch 7/1000
5/5 [==============================] - 8s 2s/step - loss: 1.5073 - val_loss: 1.6350
Epoch 8/1000
5/5 [==============================] - 7s 1s/step - loss: 1.3700 - val_loss: 1.3494
Epoch 9/1000
5/5 [==============================] - 7s 1s/step - loss: 1.5136 - val_loss: 1.1917
Epoch 10/1000
5/5 [==============================] - 7s 1s/step - loss: 0.7726 - val_loss: 1.4236
Epoch 11/1000
5/5 [======================

Traceback (most recent call last):
  File "/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_8650/2308858802.py", line 2, in <module>
    history = model.fit(x = train_gen, validation_data = val_gen, epochs = 1000,
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/keras/src/engine/training.py", line 1742, in fit
    tmp_logs = self.train_function(iterator)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ale/miniconda3/envs/tf2/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function
    return step_function(self, iter

In [ ]:
#new loss temp 0.5 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/1000
5/5 [==============================] - 53s 2s/step - loss: 2.1761 - val_loss: 2.1682
Epoch 2/1000
5/5 [==============================] - 7s 1s/step - loss: 2.1987 - val_loss: 2.1323
Epoch 3/1000
5/5 [==============================] - 7s 1s/step - loss: 2.0022 - val_loss: 2.2110
Epoch 4/1000
5/5 [==============================] - 7s 1s/step - loss: 1.9296 - val_loss: 2.1212
Epoch 5/1000
5/5 [==============================] - 6s 1s/step - loss: 2.0606 - val_loss: 1.8569
Epoch 6/1000
5/5 [==============================] - 6s 1s/step - loss: 1.8058 - val_loss: 1.9423
Epoch 7/1000
5/5 [==============================] - 6s 1s/step - loss: 1.9257 - val_loss: 1.8536
Epoch 8/1000
5/5 [==============================] - 6s 1s/step - loss: 1.8301 - val_loss: 1.8528
Epoch 9/1000
5/5 [==============================] - 6s 1s/step - loss: 1.5873 - val_loss: 1.7474
Epoch 10/1000
5/5 [==============================] - 6s 1s/step - loss: 1.4874 - val_loss: 1.3936
Epoch 11/1000
5/5 [=========

KeyboardInterrupt: 

In [ ]:
#new loss temp 0.25 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/10


2023-08-18 20:48:00.156139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-18 20:48:02.703696: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xd545e540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-18 20:48:02.703723: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2023-08-18 20:48:02.707314: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-18 20:48:02.837148: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 56s 2s/step - loss: 2.0980 - val_loss: 2.0552
Epoch 2/10
5/5 [==============================] - 7s 1s/step - loss: 2.1308 - val_loss: 2.0717
Epoch 3/10
5/5 [==============================] - 7s 2s/step - loss: 2.0044 - val_loss: 2.0857
Epoch 4/10
5/5 [==============================] - 7s 1s/step - loss: 2.0138 - val_loss: 2.0181
Epoch 5/10
5/5 [==============================] - 7s 1s/step - loss: 1.9101 - val_loss: 1.9454
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 1.8777 - val_loss: 1.9571
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 1.7303 - val_loss: 1.8144
Epoch 8/10
5/5 [==============================] - 7s 1s/step - loss: 1.7162 - val_loss: 1.8798
Epoch 9/10
5/5 [==============================] - 7s 1s/step - loss: 1.6060 - val_loss: 1.7249
Epoch 10/10
5/5 [==============================] - 7s 1s/step - loss: 1.6063 - val_loss: 1.6835


In [ ]:
#new loss temp 2 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/10
5/5 [==============================] - 54s 2s/step - loss: 2.8142 - val_loss: 2.6870
Epoch 2/10
5/5 [==============================] - 7s 1s/step - loss: 2.3177 - val_loss: 2.4348
Epoch 3/10
5/5 [==============================] - 7s 1s/step - loss: 2.2442 - val_loss: 2.4144
Epoch 4/10
5/5 [==============================] - 7s 1s/step - loss: 1.7590 - val_loss: 1.9041
Epoch 5/10
5/5 [==============================] - 7s 2s/step - loss: 1.5088 - val_loss: 2.0558
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 2.0583 - val_loss: 1.9200
Epoch 7/10
5/5 [==============================] - 7s 2s/step - loss: 1.6325 - val_loss: 1.6861
Epoch 8/10
5/5 [==============================] - 7s 2s/step - loss: 1.6393 - val_loss: 1.4117
Epoch 9/10
5/5 [==============================] - 7s 2s/step - loss: 1.0272 - val_loss: 1.5840
Epoch 10/10
5/5 [==============================] - 7s 1s/step - loss: 1.2461 - val_loss: 1.4360


In [ ]:
#new loss temp 4 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/10
5/5 [==============================] - 62s 2s/step - loss: 3.7344 - val_loss: 4.8829
Epoch 2/10
5/5 [==============================] - 7s 2s/step - loss: 5.7548 - val_loss: 4.3713
Epoch 3/10
5/5 [==============================] - 7s 2s/step - loss: 2.4620 - val_loss: 2.8179
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 3.8075 - val_loss: 4.0866
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 2.4832 - val_loss: 2.9453
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 2.5919 - val_loss: 2.4758
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 2.5942 - val_loss: 3.2169
Epoch 8/10
5/5 [==============================] - 7s 1s/step - loss: 2.4652 - val_loss: 2.2773
Epoch 9/10
5/5 [==============================] - 7s 1s/step - loss: 2.8466 - val_loss: 3.0042
Epoch 10/10
5/5 [==============================] - 7s 1s/step - loss: 2.7859 - val_loss: 2.9617


wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#new loss temp 0.25 
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 10, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

In [ ]:
%pip install tensorflow

/bin/bash: /home/ale/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#old loss loss
history = model.fit(x = train_gen, validation_data = val_gen, epochs = 1000, 
                    callbacks = [early_stopping,wandb_callback],
                   workers=4)

Epoch 1/1000


2023-08-17 18:53:48.303638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - ETA: 0s - loss: 2.0970

2023-08-17 18:54:34.005916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 47s 47s/step - loss: 2.0970 - val_loss: 2.0868
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 1.9120 - val_loss: 2.0741
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 1.6477 - val_loss: 2.0579
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4591 - val_loss: 2.0625
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 1.1526 - val_loss: 2.0743
Epoch 6/1000


...

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.3 seconds.), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 4.4 seconds.), retrying request
wandb: 429 encountered (Filestream rate limit exceeded, retrying in 8.5 seconds.), retrying request


1/1 [==============================] - 2s 2s/step - loss: 0.3328 - val_loss: 2.0694
Epoch 30/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3233 - val_loss: 2.0707
Epoch 31/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3263 - val_loss: 2.0723
Epoch 32/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3174 - val_loss: 2.0716
Epoch 33/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3193 - val_loss: 2.0788
Epoch 34/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3112 - val_loss: 2.0808
Epoch 35/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3146 - val_loss: 2.0831
Epoch 36/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3132 - val_loss: 2.0890
Epoch 37/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3022 - val_loss: 2.0948
Epoch 38/1000
1/1 [==============================] - 2s 2s/step - loss: 0.3072 - val_loss: 2.0950
Epoch 39/1000
1/1 [===============

wandb: 429 encountered (Filestream rate limit exceeded, retrying in 2.4 seconds.), retrying request


1/1 [==============================] - 2s 2s/step - loss: 0.2956 - val_loss: 2.0953
Epoch 46/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2949 - val_loss: 2.0947
Epoch 47/1000
1/1 [==============================] - 2s 2s/step - loss: 0.2924 - val_loss: 2.0950
Epoch 48/1000


KeyboardInterrupt: 

In [ ]:
import gc
gc.collect()

12390

In [ ]:
test = 1

In [ ]:
# !mkdir -p {project_location}/weights
# model.save(f"{project_location}/weights/{model_name}.h5")

In [ ]:
# wandb.save(f"{project_location}/weights/{model_name}.h5", base_path='weights/')

In [ ]:
artifact_dir

NameError: name 'artifact_dir' is not defined

In [ ]:
import wandb
import shutil
import os
run = wandb.init()
artifact = run.use_artifact('calonca/clip-flick/dataset:v0', type='dataset')
artifact_dir = artifact.download()
shutil.move(os.path.join(artifact_dir,'resized_train.zip'),os.path.join('dataset','resized_train.zip'))


wandb: Downloading large artifact dataset:v0, 243.32MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.6


'dataset/resized_train.zip'

In [ ]:
artifact = wandb.Artifact(model_name, type='dataset')
artifact.add_file(f"{project_location}/dataset/resized_train.zip")
run.log_artifact(artifact)
run.finish()